## Project Overview

### Car and Turn Profiles

We will consider 2 kinds of cars
* BMW M3
* Chevrolet Cavalier

In [1]:
from dataclasses import dataclass
from enum import Enum
import math
import numpy as np
from typing import Dict, Tuple

class PositionType(Enum):
  REAR = 1
  FRONT = 2

@dataclass
class Car:
  name: str           # name of the car
  a: float            # length from the center to the front tire (m)
  b: float            # length from the center to the rear tire (m)
  mass: float         # mass of the car (kg)
  C: Dict[PositionType, float]  # cornering stiffness (N/rad/axle)
  Iz: float           # yaw moment of inertia (kg*m^2)

generic_car = Car("some name", 1, 1.5, 123, { PositionType.FRONT: 123, PositionType.REAR: 124125 }, 87101)

class BMWM3(Car):
  def __init__(self) -> None:
   super().__init__(
        "BMW M3",
          1.36,
          1.37,
          1549,
          { PositionType.FRONT: 194000, PositionType.REAR: 240000 },
          2886
     )

bmw_car = BMWM3()

class ChevCav(Car):
  def __init__(self) -> None:
    super().__init__(
        "Chevrolet Cavalier",
          0.98,
          1.66,
          1187,
          { PositionType.FRONT: 58000, PositionType.REAR: 58000 },
          1928
    )

We will consider 2 kinds of turn profiles
* U-turn
* Lane change

In [2]:
class TurnType(Enum):
  U_TURN = 1
  LANE_CHANGE = 2

@dataclass
class TurnParams:
  turnType: TurnType    # type of turn being simulated
  startTime: float      # turn start
  delta0: float         # turn magnitude
  t0: float             # related to turn duration
  endTime: float        # related to turn duration

example_u_turn = TurnParams(TurnType.U_TURN, 1, 0.1, 2, 8)
example_lane_change = TurnParams(TurnType.LANE_CHANGE, 1, 0.1, 1/(2*math.pi), 2)

For a given turn profile, we need to be able to compute the turn angle of the front tire

In [3]:
import plotly.graph_objects as go
def delta(t: float, turn: TurnParams) -> float:
  """
  Computes the turn angle based on type of turn (U-turn or lane change).

  Inputs:
  -------
  t       : time at which turn angle is evaluated
  turn    : parameters to describe the type of turn

  Output:
  -------
  d: the turn angle (rad) at time t
  """
  if turn.turnType == TurnType.U_TURN:
    # Calculate the U-Turn here
    if t < turn.startTime:
      d = 0
    elif t >= turn.startTime and t < (turn.startTime + turn.t0):
      d = 0.5 * turn.delta0 * (1-math.cos(math.pi*(t-turn.startTime)/turn.t0))
    elif t >= (turn.startTime + turn.t0) and t < turn.endTime:
      d = turn.delta0
    elif t >= turn.endTime and t < (turn.endTime+turn.t0):
      d = 0.5 * turn.delta0 * (1-math.cos(math.pi*(turn.endTime+turn.t0-t)/turn.t0))
    elif t >= (turn.endTime+turn.t0):
      d = 0
  elif turn.turnType == TurnType.LANE_CHANGE:
    # Calculate the lane change
    if t < turn.startTime:
      d = 0
    elif t >= turn.startTime and t < (turn.startTime+2*math.pi*turn.t0):
      d = turn.delta0*math.sin((t-turn.startTime)/turn.t0)
    elif t >= (turn.startTime+2*math.pi*turn.t0):
      d = 0
  else:
    raise NotImplementedError(f"Turn type {turn.turnType} is not implemented")
  return d
  #pass # TODO: This is Homework 4 Problem 1
t = np.linspace(0,12,1000)
d = np.zeros(1000)
turn = TurnParams(TurnType.U_TURN,1,0.1,2,8)
i = 0
while i < 1000:
  d[i]= delta(t[i],turn)
  i = i+1
go.Figure(
  data=[
    go.Scatter(x=t, y=d, name="d vs t", mode="markers+lines")
  ],
  layout=go.Layout(xaxis_title="Time", yaxis_title="delta")  
)


In [4]:
t_1 = np.linspace(0,5,1000)
d_1 = np.zeros(1000)
turn = TurnParams(TurnType.LANE_CHANGE,1,0.1,1/(2*math.pi),2)
i = 0
while i < 1000:
  d_1[i]= delta(t_1[i],turn)
  i = i+1
go.Figure(
  data=[
    go.Scatter(x=t_1, y=d_1, name="d vs t", mode="markers+lines")
  ],
  layout=go.Layout(xaxis_title="Time", yaxis_title="delta")  
)

### Tire forces

We will consider two forces models:
* Linear
* Nonlinear

In [5]:
class ForceType(Enum):
  LINEAR = 1
  NONLINEAR = 2

As discussed in Lecture 4, to find the forces on the tires, we need to:
* Find the slip angle

In [6]:
import numpy as np

def slip_angle(
  a: float,
  b: float,
  d: float,
  u2: np.ndarray,
  v_centerofmass: np.ndarray,
  chiprime: float,
  position: PositionType,
) -> float:
  """
  Helper function that calculates the slip angle of the tires.

  Inputs:
  -------
  a                 : distance from center of mass to front axle (m)
  b                 : distance from center of mass to rear axle (m)
  d                 : turn angle at the current time t (radians)
  u2                : Unit vector in the car-fixed frame
  v_centerofmass    : velocity of the center of mass (m/s)
  chiprime          : dchi/dt, which is used in calculating front
                      and rear velocities (rad/s)
  position          : location of the force

  Output:
  -------
  slip_angle: Slip angle (rad) of the tire
  """
  if position == PositionType.FRONT:
    #print("1")
    slip_angle = d-math.asin(np.dot((v_centerofmass+a*chiprime*u2)/np.linalg.norm(v_centerofmass+a*chiprime*u2),u2))
  elif position == PositionType.REAR:
    # Calculate the rear tire slip angle
    slip_angle = -math.asin(np.dot((v_centerofmass-b*chiprime*u2)/np.linalg.norm(v_centerofmass-b*chiprime*u2),u2))
    #print("2")
  else:
    raise NotImplementedError(f"Position {position} not implemented")
  return slip_angle
  #pass # This is Homework 4 Problem 4
u2 = np.array([-math.sin(math.pi/8),math.cos(math.pi/8)])
v_centerofmass = np.array([100,10])
print(slip_angle(0.5, 2, math.pi/6,\
u2, v_centerofmass,\
1.5, PositionType.FRONT))
print(slip_angle(0.5, 2, math.pi/6,\
                 u2, v_centerofmass,\
                  1.5, PositionType.REAR))

0.8094692299858213
0.321357183343055


* Find the vertical force on the tire

In [7]:
G = 9.8  # acceleration due to gravity (m/s^2)

def weight_force(
    a: float, b: float, mass: float, position: PositionType
) -> float:
  """
  Helper function that calculates the weight force on the axles.

  Inputs:
  -------
  a                 : distance from center of mass to front axle (m)
  b                 : distance from center of mass to rear axle (m)
  mass              : mass of the car (kg)
  position          : location of the force

  Output:
  -------
  F: Vertical force (N) on the axle due to the car's weight
  """
  if position == PositionType.FRONT:
    # Calculate the front tire weight force
    F = (b/(a+b))*mass*G
  elif position == PositionType.REAR:
    # Calculate the rear tire weight force
    F = (a/(a+b))*mass*G
  else:
    raise NotImplementedError(f"Position {position} not implemented")
  return F
  pass # This is Homework 4 Problem 3

print(weight_force(0.5,2,100,PositionType.FRONT))
print(weight_force(0.5,2,100,PositionType.REAR))

784.0
196.0


* Use our force model

In [8]:
MU = 0.9  # coefficient of friction
class ForceType(Enum):
  LINEAR = 1
  NONLINEAR = 2
def force_model(
    forceType: ForceType, C: float, ang: float, Fn: float
) -> float:
  """
  Helper function that computes the force according to the given force model.

  Inputs:
  -------
  forceType         : type of the force model used
  C                 : cornering stiffness of the tire (N/rad)
  ang               : slip angle of the tire (rad)
  Fn                : vertical force on tire due to car's weight (N)
  
  Output:
  -------
  F: Force exerted by the road on the tires (N)
  """
  if forceType == ForceType.LINEAR:
    # Calculate the linear force
    F = C*ang
  elif forceType == ForceType.NONLINEAR:
    # Calculate the nonlinear force
    if ang == 0:
      lamda = 10
    else:
      lamda = abs(MU*Fn/(2*C*math.tan(ang)))
    if lamda < 1:
      f_lamda = (2-lamda)*lamda
    elif lamda >= 1:
      f_lamda = 1
    F = C*math.tan(ang)*f_lamda
  else:
    raise NotImplementedError(f"Force type {forceType} not implemented")
  return F
  pass # This is Homework 4 Problem 5
# Plot F vs alpha for both force models
def plot_force(car: Car) -> go.Figure:
  angles = np.linspace(0, np.pi / 40, 100)
  fig = go.Figure(
    data=[],
    layout={
      "title_text": car.name,
      "xaxis_title": "Slip Angle (degrees)",
      "yaxis_title": "Force (N)",
      }, 
    )
  for forceType in [ForceType.LINEAR, ForceType.NONLINEAR]: 
    for position in [PositionType.FRONT, PositionType.REAR]:
        # force calculation for each angle in angles array
        Fn = weight_force(car.a, car.b, car.mass, position)
        forces = np.array([
          force_model(forceType, car.C[position], angle, Fn)
          for angle in angles 
          ])
                
        fig.add_trace(
                  go.Scatter(
          name=f"{forceType.name} {position.name}",
          x=180 / np.pi * angles,
          y=forces,
          ) )
  return fig 
plot_force(BMWM3())

* Find the force using the slip angle, weight force, and force model

In [9]:
def force(
  car: Car,
  d: float,
  u2: np.ndarray,
  v_centerofmass: np.ndarray,
  chiprime: float,
  position: PositionType,
  forceType: ForceType,
) -> float:
  """
  Calculates the force exerted by the road on the tires.
  The function computes using either linear or nonlinear methods

  Inputs:
  -------
  car               : car parameters
  d                 : turn angle at the current time t (radians)
  u2                : Unit vector in the car-fixed frame
  v_centerofmass    : velocity of the center of mass (m/s)
  chiprime          : dchi/dt, which is used in calculating front
                      and rear velocities (rad/s)
  position          : location of the force
  forceType         : type of the force model used

  Output:
  -------
  F: Force exerted by the road on the tires (N)
  """
  # Need to call slip_angle(...)
  slip = slip_angle(car.a,car.b,d,u2,v_centerofmass,chiprime,position)
  # Need to call weight_force(...)
  FN = weight_force(car.a,car.b,car.mass,position)
  # Need to call force_model(...)
  F = force_model(forceType,car.C[position],slip,FN)
  return F
  pass # This is Homework 4 Problem 6

In [10]:
# Setup some test parameters
bmw = BMWM3()
v_com = np.array([10, 7])
u2 = np.array([-np.sin(0.5), np.cos(0.5)])
chiprime = 4 # rad/s

In [11]:
# Force as a function of delta
import numpy as np 
deltas = np.linspace(-np.pi/3, np.pi/3, 1000)
f = np.array([
    force(bmw,d,u2,v_com,chiprime,PositionType.FRONT,ForceType.LINEAR)
    for d in deltas
])

go.Figure(
  data=[
  go.Scatter(x=deltas, y=f)
  ],
  layout=go.Layout(
  xaxis_title="$\delta$ (rad)",
  yaxis_title="Force (N)"
  )
  )


In [12]:
# Force as a function of chiprime
d = np.pi/6
chiprimes = np.linspace(-10, 10, 1000)
f = np.array([
  force(bmw, d, u2, v_com, chiprime, PositionType.FRONT, ForceType.LINEAR)
for chiprime in chiprimes ])
go.Figure(
  data=[
    go.Scatter(x=chiprimes, y=f)
  ],
  layout=go.Layout(
    xaxis_title="chiprime",
    yaxis_title="Force (N)"
) )

In [13]:
# Force as a function of center of mass velocity
chiprime = 5
vxs = np.linspace(1, 20, 1000)
f = np.array([
  force(bmw, d, u2, np.array([vx, v_com[1]]), chiprime, PositionType.FRONT, ForceType.LINEAR)
for vx in vxs ])
go.Figure(
  data=[
    go.Scatter(x=vxs, y=f)
  ],
  layout=go.Layout(
      xaxis_title="v_x",
    yaxis_title="Force (N)"
  )
)

In [14]:
# Force on rear tire as a function of chiprime
chiprimes = np.linspace(-40, 20, 1000)
f = np.array([
  force(bmw, d, u2, v_com, chiprime, PositionType.REAR, ForceType.LINEAR)
for chiprime in chiprimes ])
go.Figure(
  data=[
    go.Scatter(x=chiprimes, y=f)
  ],
  layout=go.Layout(
    xaxis_title="chiprime",
    yaxis_title="Force (N)"
) )

In [15]:
# Nonlinear force as a function of delta
chiprime = 4
deltas = np.linspace(-np.pi/3, np.pi/3, 1000)
f = np.array([
  force(bmw, d, u2, v_com, chiprime, PositionType.FRONT, ForceType.NONLINEAR)
for d in deltas ])
go.Figure(
  data=[
    go.Scatter(x=deltas, y=f)
  ],
  layout=go.Layout(
    xaxis_title="$\delta$",
    yaxis_title="Force (N)"
) )

In [16]:
# Test force function
u2 = np.array([-math.sin(math.pi/8),math.cos(math.pi/8)])
v_centerofmass = np.array([50,10])
for car in [BMWM3(),ChevCav()]:
  for force_type in [ForceType.LINEAR,ForceType.NONLINEAR]:
    for position in [PositionType.FRONT,PositionType.REAR]:
      print(force(car, math.pi/6, u2, v_centerofmass, 0.1, position, force_type))

138959.18819653985
47505.08499494652
6786.526992650096
6565.48287841993
41586.939391685366
11512.71759777082
6368.7342400350335
3562.686484684795


### Motion dynamics

In [17]:
def motion_matrix(
  t: float,
  mm: np.ndarray,
  car: Car,
  forceType: ForceType,
  turn: TurnParams
) -> np.ndarray:
  """
  All constants and variables are defined initially. Then, the
  column vector (mm_prime) representing the derivatives is
  defined using column vector (mm) and the equations of motions.
  A helper function (Force) is used to calculate the
  force on the tires.

  Inputs:
  -------
  t         : time at which the ODE is evaluated (s)
  mm        : Column vector with the following values at time t:
              [   x
                  y
                  chi
                  dx/dt
                  dy/dt
                dchi/dt ]
  car       : car parameters
  forceType : type of the force model used
  turn      : turn parameters

  Output:
  --------
  mm_prime : Column vector with the following values at time t:
            [        dx/dt
                      dy/dt
                    dchi/dt
              d(xprime)/dt
              d(yprime)/dt
            d(chiprime)/dt ]
  """
  # Need to call delta(...)
  d = delta(t,turn)
  # Need to call force(...) on both tires
  v_centerofmass = mm[3:5]
  chi = mm[2]
  chiprime = mm[5]
  u2 = np.array([-math.sin(chi),math.cos(chi)])
  Ff = force(car,d,u2,v_centerofmass,chiprime,PositionType.FRONT,forceType)
  Fr = force(car,d,u2,v_centerofmass,chiprime,PositionType.REAR,forceType)
  # Need to compute the total force
  # Need to compute the total torque
  return np.array([
        mm[3],
        mm[4],
        mm[5],
        1/car.mass*(-Fr*math.sin(chi)-Ff*math.sin(chi+d)),
        1/car.mass*(Fr*math.cos(chi)+Ff*math.cos(chi+d)),
        1/car.Iz*(car.a*Ff*math.cos(d)-car.b*Fr)
  ]) 

  pass # This will be on Homework 5

In [18]:
mm = np.array([0, 0, 0.5, 10, 1, 4])
turn = TurnParams(TurnType.U_TURN, 0.1, np.pi/24, 1, 5.45) 
for car in BMWM3(), ChevCav():
  for forceType in ForceType.LINEAR, ForceType.NONLINEAR: 
    mmprime = motion_matrix(0.6, mm, car, forceType, turn) 
    for mmp in mmprime:
      print(f"{mmp:15.10f}", end = '') 
    print("")


  10.0000000000   1.0000000000   4.0000000000 -52.3491866960  97.4934239447 -99.2543697396
  10.0000000000   1.0000000000   4.0000000000   0.0658760823   0.4287870782  -6.1415906364
  10.0000000000   1.0000000000   4.0000000000 -21.6394619623  39.1768877727 -40.5818287463
  10.0000000000   1.0000000000   4.0000000000  -3.2245730231   5.4753175741  -1.4115268890


In [19]:
def plot_mmprime(
 xlabel: str,
 xval: np.ndarray,
 mmprime: np.ndarray
 ) -> go.Figure:
 """
 Plots mmprime

 Inputs:
 as a function of the given x variable.
 -------
 xlabel :
 xval :
 mmprime :

 Output:
 --------
 fig : Figure for visualizing mmprime
 """

 fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
 fig.add_trace(
 go.Scatter(x=xval, y=mmprime[:,3], name="vx'"),
 row=1, col=1
 )
 fig.add_trace(
 go.Scatter(x=xval, y=mmprime[:,4], name="vy'"),
 row=1, col=1
 )
 fig.add_trace(
 go.Scatter(x=xval, y=mmprime[:,5], name="chi'"),
 row=2, col=1
 )

 fig.update_xaxes(title_text=xlabel, row=2, col=1) 
 return fig

In [20]:
from plotly.subplots import make_subplots
# mmprime as a function of time
mm = np.array([0, 0, 0.5, 10, 1, 4])
turn = TurnParams(TurnType.U_TURN, 0.1, np.pi/24, 1, 5.45)
bmw = BMWM3()
times = np.linspace(0, 7, 1000)
mmprime = np.stack([
  motion_matrix(t, mm, bmw, ForceType.NONLINEAR, turn) 
  for t in times
])

plot_mmprime("Time (s)", times, mmprime)


In [21]:
# mmprime as a function of chi
t = 0.6
chis = np.linspace(-1.4, 1.6, 1000) 
mmprime = np.empty((1000, 6))
for i, chi in enumerate(chis):
  mm[2] = chi
  mmprime[i, :] = motion_matrix(t, mm, bmw, ForceType.NONLINEAR, turn)
plot_mmprime("Chi", chis, mmprime)

In [22]:
# mmprime as a function of center of mass velocity
mm = np.array([0, 0, 0.5, 10, 1, 4])
vxs = np.linspace(1, 20, 1000)
mmprime = np.empty((1000, 6))
for i, vx in enumerate(vxs):
  mm[3] = vx
  mmprime[i, :] = motion_matrix(t, mm, bmw, ForceType.NONLINEAR, turn)
plot_mmprime("vx", vxs, mmprime)


In [23]:
# mmprime as a function of chiprime
mm = np.array([0, 0, 0.5, 10, 1, 4])
chiprimes = np.linspace(-10, 10, 1000)
mmprime = np.empty((1000, 6))
for i, chiprime in enumerate(chiprimes):
  mm[5] = chiprime
  mmprime[i, :] = motion_matrix(t, mm, bmw, ForceType.NONLINEAR, turn)
plot_mmprime("chiprime", chiprimes, mmprime)

In [24]:
from scipy.integrate import solve_ivp
def solve_motion(
  t_span: Tuple[float, float],
  dt: float,
  mm0: np.ndarray,
  car: Car,
  forceType: ForceType,
  turn: TurnParams
) -> Tuple[np.ndarray, np.ndarray]:
  """
  Solves for the car motion.

  Inputs:
  -------
  t_span    : time range over which to solve the ODE
  dt        : time step interval
  mm0       : initial car motion (see column vector format in motion_matrix)
  car       : car parameters
  forceType : type of the force model used
  turn      : turn parameters

  Output:
  --------
  times     : time points for the solution (has shape (T,))
  motion    : value of solution at the time points (has shape (6, T))
  """
  # Need to call solve_ivp(...)
  #  - Need to use the motion_matrix
  #  - Need to use solve_ivp's "args" parameter
  # Time span for simulation
  times = np.arange(t_span[0], t_span[1] + 1e-6, dt)
  # Numerically solve the ODEs
  soln = solve_ivp(motion_matrix,t_span,mm0,method = 'RK45',rtol=1e-6,atol=1e-6,args=(car,forceType,turn))
  # Fill in the missing arguments here
  return soln.t, soln.y
  pass # This will be on Homework 5

In [25]:
u_turn = TurnParams(TurnType.U_TURN,0.1,math.pi/24,1,5.45)
lc_turn = TurnParams(TurnType.LANE_CHANGE,0.1,math.pi/50,0.5,0.1+2*math.pi*0.5)
t_span = (0,8)
dt = .01
mm0 = np.array([0,0,0,30,0,0])
mm0[3] *= 1609.344 / 3600

In [26]:
for turn in lc_turn, u_turn: 
  for car in BMWM3(), ChevCav():
    for forceType in ForceType.LINEAR, ForceType.NONLINEAR:
      times, motion = solve_motion(t_span, dt, mm0, car, forceType, turn) 
      for m in motion[:,-1]:
        print(f"{m:15.10f}", end = '') 
      print("")


 102.8644412396   5.8008621107   0.0008017603  12.9195465423   0.0103592376   0.0000000399
 102.8644473639   5.8008923383   0.0008015670  12.9195490945   0.0103568457   0.0000000002
 102.6834195959   4.7023642621   0.0006475986  12.8541497428   0.0083227029   0.0000001938
 102.6832285687   4.7031568882   0.0006487758  12.8541404289   0.0083387738   0.0000000760
 -15.5093378247  45.8221026882   3.0597341544 -12.0054955036   0.9849522916   0.0000002503
 -10.8259139000  49.1132929136   2.9445844511 -11.5554612141   2.3064372625   0.0000002004
   8.1411956807  60.5079633019   2.4643221568  -8.7719261403   7.0540048599  -0.0000000427
  10.3295982369  61.0166143180   2.4222340269  -8.4401844722   7.3930429126  -0.0000001011


In [27]:
class ViewType(Enum):
  DISTANT = 1
  ZOOM = 2

@dataclass
class ViewParams:
  type: ViewType
  xmin: float
  xmax: float
  ymin: float
  ymax: float

In [28]:
import plotly.graph_objects as go

def animate_car(
  times: np.ndarray,
  motion: np.ndarray,
  car: Car,
  turn: TurnParams,
  view: ViewParams
) -> go.Figure:
  """
  Plots the car motion.

  Inputs:
  -------
  times     : time points for the solution (has shape (T,))
  motion    : value of solution at the time points (has shape (6, T))
  mm0       : initial car motion (see column vector format in motion_matrix)
  car       : car parameters
  turn      : turn parameters
  view      : animation parameters

  Output:
  --------
  fig       : animation figure
  """
  pass # This code will be given to you

In [29]:
car = BMWM3()
forceType = ForceType.LINEAR
turn = TurnParams(...)

t_span = (0, 12)
dt = 0.01
mm0 = np.array([0, 0, 0, 2, 0, 0])

times, motion = solve_motion(t_span, dt, mm0, car, forceType, turn)

view = ViewParams(ViewType.DISTANT, ...)

animate_car(times, motion, mm0, car, turn, view)

TypeError: ignored

BMW linear U-turn

In [ ]:
from dataclasses import dataclass
from enum import Enum
from typing import Dict

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.integrate import solve_ivp

class TurnType(Enum):
  U_TURN = 1
  LANE_CHANGE = 2

@dataclass
class TurnParams:
  turnType: TurnType    # type of turn being simulated
  startTime: float      # turn start
  delta0: float         # turn magnitude
  t0: float             # related to turn duration
  endTime: float        # related to turn duration

class PositionType(Enum):
  REAR = 1
  FRONT = 2

class ForceType(Enum):
  LINEAR = 1
  NONLINEAR = 2

@dataclass
class Car:
  name: str           # name of the car
  a: float            # length from the center to the front tire (m)
  b: float            # length from the center to the rear tire (m)
  mass: float         # mass of the car (kg)
  C: Dict[PositionType, float]  # cornering stiffness (N/rad/axle)
  Iz: float           # yaw moment of inertia (kg*m^2)

In [ ]:
def plot_trajectory(times: np.ndarray, motion: np.ndarray, title_string: str) -> go.Figure:
  """
  Plot the trajectory of the car.

  Inputs:
  -------
  times: time points for the solution (has shape (T,))
  motion: value of solution at the time points (has shape (6, T))
  title_string  : title of the plot
  
  Output:
  --------
  fig: trajectory figure
  """
  pass #this is Homework 6 problem 1 

def solve_motion(
t_span= (0,8) # t = (tmin, tmax) : interval over which the solution has to be computed (s)
dt= .01 #dt: time step (s)
mm0 = np.array([0,0,0,30,0,0]) #initial condition row vector with the following values at time t=0:[x(0) y(0) chi(0) dx(0)/dt dy(0)/dt dchi(0)/dt]
mm0[3]*=1609.344 / 3600 # Convert units of initial velocity from mph to m/s


plot_trajectory(times, motion,"BMW linear U turn")